<a href="https://colab.research.google.com/github/Khadijah911/smart_hotel_chatbot/blob/main/smart_hotel_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain_community.document_loaders import TextLoader
#from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [ ]:
from langchain_community.document_loaders import TextLoader
#from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS

In [ ]:
loader = TextLoader("hotel_info.txt", encoding="utf-8")
docs = loader.load()

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=openai_api_key)

In [ ]:
from openai import OpenAI
import os
from google.colab import userdata
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# Create embeddings and vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=openai_api_key)
vectorstore = FAISS.from_documents(chunks, embeddings)


# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
rag_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "how much is spaghetti"
result = rag_chain.invoke({"question":query})
print(result["answer"])

The Spaghetti Bolognese costs ₦3000.


In [ ]:
def hotel_chat(message, history):
    # Check if the message is about rooms (SQL)
    if "room" in message.lower() or "available" in message.lower() or "price" in message.lower():
        response = sql_retrieval(message)
    else:
        # Just pass the question as a string
        rag_response = rag_chain.invoke(message)
        response = rag_response["answer"]

    return response

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
rag_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)
def chat(message, history):
    result = rag_chain.invoke({"question": message})
    return result["answer"]

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5343c5bac198b020a8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import sqlite3

# Create database
conn = sqlite3.connect('hotel.db')
cursor = conn.cursor()

# Create rooms table
cursor.execute('''
CREATE TABLE rooms (
    room_id INTEGER PRIMARY KEY,
    room_number TEXT,
    room_type TEXT,
    price REAL,
    status TEXT,
    description TEXT,
    amenities TEXT
)
''')

# Insert sample data
rooms_data = [
    (1, '101', 'Standard', 100.0, 'available', 'Cozy standard room with city view', 'WiFi, TV, AC'),
    (2, '102', 'Deluxe', 150.0, 'available', 'Spacious deluxe room with king bed', 'WiFi, TV, AC, Mini-bar'),
    (3, '201', 'Suite', 250.0, 'occupied', 'Luxury suite with ocean view', 'WiFi, TV, AC, Mini-bar, Balcony'),
    (4, '202', 'Standard', 100.0, 'available', 'Comfortable room near elevator', 'WiFi, TV, AC'),
    (5, '301', 'Deluxe', 150.0, 'available', 'Top floor deluxe with mountain view', 'WiFi, TV, AC, Mini-bar')
]

cursor.executemany('INSERT INTO rooms VALUES (?,?,?,?,?,?,?)', rooms_data)
conn.commit()
conn.close()

print("Database created!")

Database created!


In [ ]:
db_name = "hotel.db"

In [ ]:


from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///hotel.db")




In [ ]:
db.get_usable_table_names()




['rooms']

In [ ]:
print(db.get_table_info())



CREATE TABLE rooms (
	room_id INTEGER, 
	room_number TEXT, 
	room_type TEXT, 
	price REAL, 
	status TEXT, 
	description TEXT, 
	amenities TEXT, 
	PRIMARY KEY (room_id)
)

/*
3 rows from rooms table:
room_id	room_number	room_type	price	status	description	amenities
1	101	Standard	100.0	available	Cozy standard room with city view	WiFi, TV, AC
2	102	Deluxe	150.0	available	Spacious deluxe room with king bed	WiFi, TV, AC, Mini-bar
3	201	Suite	250.0	occupied	Luxury suite with ocean view	WiFi, TV, AC, Mini-bar, Balcony
*/


In [ ]:
# SQL query chain
from langchain.chains import create_sql_query_chain
chain = create_sql_query_chain(llm, db)

In [ ]:
response = chain.invoke({"question": "What tables are available in this database?"})
print(response)

SQLQuery: SELECT "name" FROM sqlite_master WHERE "type" = 'table';


In [ ]:
import re

# Ask a question
response = chain.invoke({"question": "What rooms are available?"})
print("Full LLM output:\n", response)

# Extract only the SQL query part using regex
match = re.search(r"SELECT.*;", response, re.DOTALL)
if match:
    sql_query = match.group(0)
    print("\nExtracted SQL Query:\n", sql_query)
else:
    raise ValueError("No valid SQL query found in the response.")

# Runing the SQL query on my database
result = db.run(sql_query)
print("\nQuery Result:")
print(result)


Full LLM output:
 SQLQuery: ```SELECT "room_number", "room_type", "price", "description", "amenities" FROM rooms WHERE "status" = 'available' LIMIT 5;```

Extracted SQL Query:
 SELECT "room_number", "room_type", "price", "description", "amenities" FROM rooms WHERE "status" = 'available' LIMIT 5;

Query Result:
[('101', 'Standard', 100.0, 'Cozy standard room with city view', 'WiFi, TV, AC'), ('102', 'Deluxe', 150.0, 'Spacious deluxe room with king bed', 'WiFi, TV, AC, Mini-bar'), ('202', 'Standard', 100.0, 'Comfortable room near elevator', 'WiFi, TV, AC'), ('301', 'Deluxe', 150.0, 'Top floor deluxe with mountain view', 'WiFi, TV, AC, Mini-bar')]


In [ ]:

def sql_retrieval(question):
    response = chain.invoke({"question": question})

    # Extract SQL query
    match = re.search(r"SELECT.*;", response, re.DOTALL)
    if match:
        sql_query = match.group(0)
    else:
        raise ValueError("No valid SQL query found in the response.")

    # Execute query
    result = db.run(sql_query)

    # Generate a friendly answer
    answer_prompt =  f"""You are a helpful hotel assistant. you are responding to the hotel's customer,
    who are asking you about rooms
    You MUST always respond in English, .
    Be friendly, clear, and professional.
    you can respond to their greetings with ,welcome to labama,how may i help you.


    The user asked: '{question}'
    The database returned: {result}
    Create a friendly and clear response."""

    final_answer = llm.invoke(answer_prompt)
    return final_answer.content






In [ ]:
sql_retrieval("What rooms are available for a price of 100.0?")

'Welcome to Labama! How may I help you? \n\nI’m happy to inform you that we have two rooms available for a price of $100.00:\n\n1. **Room 101** - This is a cozy standard room with a lovely city view.\n2. **Room 202** - A comfortable standard room conveniently located near the elevator.\n\nIf you’d like to know more about either room or if you’d like to book one, please let me know!'

In [ ]:
import gradio as gr

def chat(message, history):
    return sql_retrieval(message)

app = gr.ChatInterface(
    fn=chat,
    title="hotel lamaba guide",
    description="Ask me anything about the hotel. i will do well to provide you with answers!"
)

app.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://289fe4cb507b48e1bf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
def hotel_chat(message, history):


    # Check if the message is about rooms (SQL)
    if "room" in message.lower() or "available" in message.lower() or "price" in message.lower():
        response = sql_retrieval(message)
    else:
        # Otherwise use RAG
        result = rag_chain.invoke({"question": message})
        return result["answer"]

    return response

In [ ]:
demo = gr.ChatInterface(
    fn=hotel_chat,
    title="🏨 Smart Hotel Assistant",
    description="Ask about rooms, prices, or general hotel info like food, services, and contact.",
    type="messages"
)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4bcc74a9aa65a7ebad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import requests

# Pushover credentials


def send_push_notification(message):
    payload = {
        "token": PUSHOVER_TOKEN,
        "user": PUSHOVER_USER,
        "message": message,
        "title": "Hotel Complaint Alert 🚨",
        "priority": 1
    }
    requests.post("https://api.pushover.net/1/messages.json", data=payload)

In [ ]:
send_push_notification('hii')

In [ ]:
def classify_query(message):
    prompt = f"""
You are a classifier for a hotel assistant.

Classify the user's message into EXACTLY ONE of these categories:

1. sql
   The message is about rooms, availability, booking, room prices, or anything requiring a database query.

2. rag
   The message asks for general hotel information such as food, amenities, services, check-in/out time, etc.

3. complaint
   The message expresses dissatisfaction, a problem, an issue, or frustration.
   Examples: "my room is dirty", "the AC is not working", "the staff was rude", "I want to report an issue".

User message: "{message}"

Your answer MUST be exactly ONE WORD:
sql
rag
complaint
"""

    result = llm.invoke(prompt)
    return  result.content.strip().upper()



In [ ]:
def hotel_chat(message, history):
    category = classify_query(message)

    if category == "COMPLAINT":
        send_push_notification(f"Complaint received: {message}")
        return "I’m really sorry to hear that 😟. I’ve sent your complaint to the front desk immediately."

    elif category == "SQL":
        return sql_retrieval(message)

    else:  # rag
        rag_response = rag_chain.invoke({"question": message})
        return rag_response["answer"]


In [ ]:
demo = gr.ChatInterface(
    fn=hotel_chat,
    title="Labama Hotel Assistant",
    description="Ask about rooms, prices,  general hotel info like food, services, and contact,you can also make your complaints here.",
    type="messages"
)

demo.launch()



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b3bc987d9809fbe08.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
